<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/upstage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在Colab中打开"/></a>


# Upstage


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-llms-upstage==0.1.0

In [ ]:
!pip install llama-index

## 基本用法


#### 使用提示调用`complete`


In [ ]:
import os

os.environ["UPSTAGE_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from llama_index.llms.upstage import Upstage

resp = Upstage().complete("Paul Graham is ")

In [ ]:
print(resp)

Paul Graham is a computer scientist, entrepreneur, and essayist. He is best known as the co-founder of the venture capital firm Y Combinator, which has funded and incubated many successful startups. He is also the author of several influential essays on entrepreneurship, startup culture, and technology.


#### 使用消息列表调用 `chat` 函数


In [ ]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.upstage import Upstage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = Upstage().chat(messages)

In [ ]:
print(resp)

assistant: I am Captain Redbeard, the fearless pirate!


## 流式处理


使用 `stream_complete` 终端点


In [ ]:
from llama_index.llms.upstage import Upstage

llm = Upstage()
resp = llm.stream_complete("Paul Graham is ")

In [ ]:
for r in resp:
    print(r.delta, end="")

Paul Graham is a computer scientist, entrepreneur, and essayist. He is best known for co-founding the startup accelerator Y Combinator, which has helped launch some of the most successful tech companies in the world, including Airbnb, Dropbox, and Stripe. He is also the author of several influential essays on startup culture and entrepreneurship, including "How to Start a Startup" and "Hackers & Painters."

使用 `stream_chat` 端点


In [ ]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.upstage import Upstage

llm = Upstage()
messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.stream_chat(messages)

In [ ]:
for r in resp:
    print(r.delta, end="")

I am Captain Redbeard, the fearless pirate!

## 配置模型


In [ ]:
from llama_index.llms.upstage import Upstage

llm = Upstage()

In [ ]:
resp = llm.complete("Paul Graham is ")

In [ ]:
print(resp)

Paul Graham is a computer scientist, entrepreneur, and essayist. He is best known for co-founding the startup accelerator Y Combinator, which has helped launch some of the most successful tech companies in the world, including Airbnb, Dropbox, and Stripe. He is also the author of several influential essays on entrepreneurship, startups, and technology, including "How to Start a Startup" and "Hackers & Painters."


In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [ ]:
print(resp)

assistant: I am Captain Redbeard, the fearless pirate!


## 函数调用

Upstage模型原生支持函数调用。这方便地与LlamaIndex工具抽象集成，让您可以将任意的Python函数插入到LLM中。


In [ ]:
from pydantic import BaseModel
from llama_index.core.tools import FunctionTool


class Song(BaseModel):
    """带有名称和艺术家的歌曲"""

    name: str
    artist: str


def generate_song(name: str, artist: str) -> Song:
    """使用提供的名称和艺术家生成一首歌曲。"""
    return Song(name=name, artist=artist)


tool = FunctionTool.from_defaults(fn=generate_song)

In [ ]:
from llama_index.llms.upstage import Upstage

llm = Upstage()
response = llm.predict_and_call([tool], "Generate a song")
print(str(response))

name='My Song' artist='John Doe'


我们也可以进行多个函数的调用。


In [ ]:
llm = Upstage()
response = llm.predict_and_call(
    [tool],
    "Generate five songs from the Beatles",
    allow_parallel_tool_calls=True,
)
for s in response.sources:
    print(f"Name: {s.tool_name}, Input: {s.raw_input}, Output: {str(s)}")

Name: generate_song, Input: {'args': (), 'kwargs': {'name': 'Beatles', 'artist': 'Beatles'}}, Output: name='Beatles' artist='Beatles'


## 异步


In [ ]:
from llama_index.llms.upstage import Upstage

llm = Upstage()

In [ ]:
resp = await llm.acomplete("Paul Graham is ")

In [ ]:
print(resp)

Paul Graham is a computer scientist, entrepreneur, and essayist. He is best known as the co-founder of the startup accelerator Y Combinator, which has helped launch and fund many successful tech companies. He is also the author of several influential essays on startups, entrepreneurship, and technology, including "How to Start a Startup" and "Hackers & Painters."


In [ ]:
resp = await llm.astream_complete("Paul Graham is ")

In [ ]:
async for delta in resp:
    print(delta.delta, end="")

Paul Graham is a computer scientist, entrepreneur, and essayist. He is best known as the co-founder of the startup accelerator Y Combinator, which has helped launch some of the most successful tech companies in the world, including Airbnb, Dropbox, and Stripe. Graham is also a prolific writer, and his essays on topics such as startup advice, artificial intelligence, and the future of work have been widely read and influential in the tech industry.

异步函数调用也是支持的。


In [ ]:
llm = Upstage()
response = await llm.apredict_and_call([tool], "Generate a song")
print(str(response))

name='My Song' artist='Me'


## 在每个实例级别设置API密钥
如果需要的话，您可以使用不同的API密钥来管理不同的LLM实例。


In [ ]:
from llama_index.llms.upstage import Upstage

llm = Upstage(api_key="BAD_KEY")
resp = Upstage().complete("Paul Graham is ")
print(resp)

Paul Graham is a computer scientist, entrepreneur, and essayist. He is best known as the co-founder of the startup accelerator Y Combinator, which has helped launch and fund many successful tech companies. He is also the author of several influential essays on startup culture and technology, including "How to Start a Startup" and "Hackers & Painters."
